In [2]:
#importing all libraries

import pandas as pd
import numpy as np 
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score


from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from dateutil.parser import parse

In [3]:
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

#Data Preprocessing

In [4]:
train_df.info()
test_df.info()

train_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80176 entries, 0 to 80175
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              80176 non-null  object 
 1   org               76725 non-null  object 
 2   tld               76723 non-null  object 
 3   ccs               80176 non-null  int64  
 4   bcced             80176 non-null  int64  
 5   mail_type         80004 non-null  object 
 6   images            80176 non-null  int64  
 7   urls              80176 non-null  int64  
 8   salutations       80176 non-null  int64  
 9   designation       80176 non-null  int64  
 10  chars_in_subject  80160 non-null  float64
 11  chars_in_body     80176 non-null  int64  
 12  label             80176 non-null  int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 8.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 34365 entries, 0 to 34364
Data columns (total 12 columns):
 #   Column    

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2
1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1
2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1
3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1
4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1


In [3]:
#function to split date column

def normalize_date(date):

    if len(date)>27:
        if len(date)>41:
            date=date[:31]
        dt=parse(date)
        day=date[:3]
        return day+' '+dt.strftime("%d %m %Y %H")


    day=""
    dt=parse(date)
    return day+' '+dt.strftime("%d %m %Y %H")

In [4]:
#Splitting date column in training data

new = train_df['date'].apply(lambda x: normalize_date(x))

new2 = new.str.split(" ", n = 1, expand = True)
train_df['day']=new2[0]
train_df['date_new']=new2[1].str[0:2]
train_df['month']=new2[1].str[3:5]
train_df['year']=new2[1].str[6:10]
train_df['hour']=new2[1].str[11:13]
train_df["hour"] = train_df["hour"].astype(int)



#Splitting date column in testing data


new3 = test_df['date'].apply(lambda x: normalize_date(x))

new4 = new3.str.split(" ", n = 1, expand = True)
test_df['day']=new4[0]
test_df['date_new']=new4[1].str[0:2]
test_df['month']=new4[1].str[3:5]
test_df['year']=new4[1].str[6:10]
test_df['hour']=new4[1].str[11:13]
test_df["hour"] = test_df["hour"].astype(int)


In [39]:
train_df.head(101)
#test_df.head(101)

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label,day,date_new,month,year,hour
0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2,Mon,06,11,2017,11
1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1,Wed,14,02,2018,11
2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1,Wed,06,07,2016,19
3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1,Fri,11,10,2019,11
4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1,Tue,07,11,2017,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Mon, 5 Nov 2018 09:35:10 +0000",centralesupelec,fr,0,0,text/html,4,14,0,0,191.0,8420,1,Mon,05,11,2018,9
97,"Wed, 31 May 2017 05:37:18 +0000",innerchef,com,0,0,multipart/alternative,5,33,0,0,70.0,43332,2,Wed,31,05,2017,5
98,"Wed, 09 Mar 2016 16:51:20 +0000",youtube,com,0,0,multipart/alternative,14,60,0,0,57.0,55206,1,Wed,09,03,2016,16
99,"Tue, 3 May 2016 17:08:35 +0530",iiitd,ac.in,0,0,multipart/alternative,0,6,1,0,30.0,2601,1,Tue,03,05,2016,17


In [18]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

date                   0
org                 3451
tld                 3453
ccs                    0
bcced                  0
mail_type            172
images                 0
urls                   0
salutations            0
designation            0
chars_in_subject      16
chars_in_body          0
label                  0
day                    0
date_new               0
month                  0
year                   0
hour                   0
dtype: int64
date                   0
org                 1536
tld                 1536
ccs                    0
bcced                  0
mail_type             80
images                 0
urls                   0
salutations            0
designation            0
chars_in_subject       6
chars_in_body          0
day                    0
date_new               0
month                  0
year                   0
hour                   0
dtype: int64


In [5]:
#populating all null values

#training data
imp = SimpleImputer(strategy='most_frequent', missing_values=np.NaN)
#Missing values in object fields replace my most frequent
train_df.mail_type= imp.fit_transform(train_df['mail_type'].values.reshape(-1,1))[:,0]
train_df.org= imp.fit_transform(train_df['org'].values.reshape(-1,1))[:,0]
train_df.tld= imp.fit_transform(train_df['tld'].values.reshape(-1,1))[:,0]
#train_df.hour= imp.fit_transform(train_df['hour'].values.reshape(-1,1))[:,0]
#train_df.time= imp.fit_transform(train_df['time'].values.reshape(-1,1))[:,0]
#train_df.date_new= imp.fit_transform(train_df['date_new'].values.reshape(-1,1))[:,0]


train_df['chars_in_subject'].fillna(train_df['chars_in_subject'].median(), inplace=True)




#test

#Missing values in object fields replace my most frequent
test_df.mail_type= imp.fit_transform(test_df['mail_type'].values.reshape(-1,1))[:,0]
test_df.org= imp.fit_transform(test_df['org'].values.reshape(-1,1))[:,0]
test_df.tld= imp.fit_transform(test_df['tld'].values.reshape(-1,1))[:,0]
#test_df.hour= imp.fit_transform(test_df['hour'].values.reshape(-1,1))[:,0]
#test_df.time= imp.fit_transform(test_df['time'].values.reshape(-1,1))[:,0]
#test_df.date_new= imp.fit_transform(test_df['date_new'].values.reshape(-1,1))[:,0]

test_df['chars_in_subject'].fillna(test_df['chars_in_subject'].median(), inplace=True)


print(train_df.isnull().sum())
print(test_df.isnull().sum())


date                0
org                 0
tld                 0
ccs                 0
bcced               0
mail_type           0
images              0
urls                0
salutations         0
designation         0
chars_in_subject    0
chars_in_body       0
label               0
day                 0
date_new            0
month               0
year                0
hour                0
dtype: int64
date                0
org                 0
tld                 0
ccs                 0
bcced               0
mail_type           0
images              0
urls                0
salutations         0
designation         0
chars_in_subject    0
chars_in_body       0
day                 0
date_new            0
month               0
year                0
hour                0
dtype: int64


In [6]:
#Splitting mail_type column for train data

temp = train_df['mail_type'].str.lower().str.strip().str.split('/', expand=True)
temp.rename(columns = {0:'mail_type_1', 1:'mail_type_2'}, inplace = True)

train_df = pd.concat([train_df, temp], axis=1)
train_df.drop('mail_type', axis=1, inplace=True)

In [7]:
#Splitting mail_type column for test data

temp2 = test_df['mail_type'].str.lower().str.strip().str.split('/', expand=True)
temp2.rename(columns = {0:'mail_type_1', 1:'mail_type_2'}, inplace = True)

test_df = pd.concat([test_df, temp2], axis=1)
test_df.drop('mail_type', axis=1, inplace=True)


In [22]:
train_df.head()

,date,org,tld,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,label,day,date_new,month,year,hour,mail_type_1,mail_type_2
0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,35,120,0,0,49.0,80027,2,Mon,06,11,2017,11,multipart,alternative
1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,1,7,0,0,107.0,2961,1,Wed,14,02,2018,11,multipart,alternative
2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,4,17,0,0,35.0,25149,1,Wed,06,07,2016,19,text,html
3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,0,0,0,0,15.0,635296,1,Fri,11,10,2019,11,multipart,mixed
4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,2,11,0,0,49.0,2355,1,Tue,07,11,2017,11,multipart,alternative


In [8]:
#creating period out of hours

train_df['period'] = pd.cut(x=train_df['hour'], bins=[-1, 6,12, 18, 24], labels = ['1', '2', '3', '4'])
test_df['period'] = pd.cut(x=test_df['hour'], bins=[-1, 6, 12, 18, 24], labels = ['1', '2', '3', '4'])


In [9]:
def one_hot_top(df,variable,top_X_labels):
  for label in top_X_labels:
    df[label] = np.where(df[variable]==label,1,0)
  df[[variable] + top_X_labels]

In [10]:
#one hot encoding of org

f_org = [x for x in train_df.org.value_counts().sort_values(ascending=False).index]
one_hot_top(train_df,'org',f_org)
one_hot_top(test_df,'org',f_org)

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

...

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

In [11]:
#one hot encoding of tld

f_tld = [x for x in train_df.tld.value_counts().sort_values(ascending=False).index]
one_hot_top(train_df,'tld',f_tld)
one_hot_top(test_df,'tld',f_tld)

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

...

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[label] = np.where(df[variable]==label,1,0)
/var/folders/vm/v8d8156d69g1lmd1t4j0zwqw0000gn/T/ipykernel_19514/3002830861.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

In [12]:
#dropping the original org and tld colums

train_df.drop('org', axis=1, inplace=True)
train_df.drop('tld', axis=1, inplace=True)


test_df.drop('org', axis=1, inplace=True)
test_df.drop('tld', axis=1, inplace=True)

In [13]:
#checking the data

print(train_df.head())
print(test_df.head())

list(train_df.columns)

                                    date  ccs  bcced  images  urls  \
0         Mon, 6 Nov 2017 11:13:45 +0100    0      0      35   120   
1        Wed, 14 Feb 2018 11:00:16 -0000    0      0       1     7   
2         Wed, 6 Jul 2016 19:53:37 +0000    0      0       4    17   
3        Fri, 11 Oct 2019 11:25:40 +0200    0      0       0     0   
4  Tue, 07 Nov 2017 11:07:18 +0000 (UTC)    1      0       2    11   

   salutations  designation  chars_in_subject  chars_in_body  label  ...  \
0            0            0              49.0          80027      2  ...   
1            0            0             107.0           2961      1  ...   
2            0            0              35.0          25149      1  ...   
3            0            0              15.0         635296      1  ...   
4            0            0              49.0           2355      1  ...   

  hosting sap.stampmail.in com.ar chtah.com  ac.kr xerox.com flickr.com  \
0       0                0      0         0    

['date',
 'ccs',
 'bcced',
 'images',
 'urls',
 'salutations',
 'designation',
 'chars_in_subject',
 'chars_in_body',
 'label',
 'day',
 'date_new',
 'month',
 'year',
 'hour',
 'mail_type_1',
 'mail_type_2',
 'period',
 'iiitd',
 'usebackpack',
 'google',
 'amazon',
 'linkedin',
 'twitter',
 'quora',
 'gmail',
 'github',
 'youtube',
 'nrsc',
 'centralesupelec',
 'topcoder',
 'coursera',
 'phpclasses',
 'medium',
 'mailer',
 'flipkartletters',
 'granular',
 'udacity',
 'nvidia',
 'standupalice',
 'mail',
 'sampark',
 'e',
 'hackerearth',
 'reply',
 'email',
 'ieee',
 'duolingo',
 'sbi',
 'kaggle',
 'news',
 'mentor',
 'olacabs',
 'newsletter',
 'slack',
 'hackerrank',
 'send',
 'accounts',
 'hackerrankmail',
 'm',
 'angel',
 'freshersworld',
 'piazza',
 'notifications',
 'redwolf',
 'academia-mail',
 'thomascook',
 'indiatimes',
 'crm',
 'statebankrewardz',
 'aamaadmiparty',
 'signalprocessingsociety',
 'glassdoor',
 'facebookmail',
 'state',
 'edx',
 'repositoryhosting',
 'usief',
 'r

#Feature Selection

In [14]:
#creating a subset of org and tld column to perform feature selection

FS_x = train_df.iloc[:,18:]
FS_y = train_df['label']
print(FS_x.head())

   iiitd  usebackpack  google  amazon  linkedin  twitter  quora  gmail  \
0      0            0       0       0         0        0      0      0   
1      0            0       0       0         0        0      0      0   
2      0            1       0       0         0        0      0      0   
3      0            0       0       0         0        0      0      0   
4      0            0       0       0         0        0      0      0   

   github  youtube  ...  hosting  sap.stampmail.in  com.ar  chtah.com  ac.kr  \
0       0        0  ...        0                 0       0          0      0   
1       0        0  ...        0                 0       0          0      0   
2       0        0  ...        0                 0       0          0      0   
3       0        0  ...        0                 0       0          0      0   
4       1        0  ...        0                 0       0          0      0   

   xerox.com  flickr.com  mail-avis-verifies.com  cisco.com  ws  
0       

In [15]:
#using logistic regression to select best features

from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(C=0.01)
model_lr.fit(FS_x, FS_y)
model_lr.coef_

model_sfm = SelectFromModel(model_lr, prefit=True)
model_sfm.transform(FS_x).shape

/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


(80176, 261)

In [16]:
newlabels = model_sfm.get_support(indices=True)

In [17]:
train_subset = FS_x.iloc[:,newlabels]
test_subset = test_df[train_subset.columns]

In [18]:
#merging the original columns with the best features

train_df=pd.concat([train_df.iloc[:,0:18],train_subset],axis=1)
test_df=pd.concat([test_df.iloc[:,0:17],test_subset],axis=1)

In [19]:
#one hot encoding of mail_type columns

f_mail_type_1 = [x for x in train_df.mail_type_1.value_counts().sort_values(ascending=False).index]
one_hot_top(train_df,'mail_type_1',f_mail_type_1)
one_hot_top(test_df,'mail_type_1',f_mail_type_1)
train_df.drop('mail_type_1',axis=1, inplace=True)
test_df.drop('mail_type_1',axis=1, inplace=True)


f_mail_type_2 = [x for x in train_df.mail_type_2.value_counts().sort_values(ascending=False).index]
one_hot_top(train_df,'mail_type_2',f_mail_type_2)
one_hot_top(test_df,'mail_type_2',f_mail_type_2)
train_df.drop('mail_type_2',axis=1, inplace=True)
test_df.drop('mail_type_2',axis=1,inplace=True)


In [20]:
print(train_df.head())
print(test_df.head())

                                    date  ccs  bcced  images  urls  \
0         Mon, 6 Nov 2017 11:13:45 +0100    0      0      35   120   
1        Wed, 14 Feb 2018 11:00:16 -0000    0      0       1     7   
2         Wed, 6 Jul 2016 19:53:37 +0000    0      0       4    17   
3        Fri, 11 Oct 2019 11:25:40 +0200    0      0       0     0   
4  Tue, 07 Nov 2017 11:07:18 +0000 (UTC)    1      0       2    11   

   salutations  designation  chars_in_subject  chars_in_body  label  ... text  \
0            0            0              49.0          80027      2  ...    0   
1            0            0             107.0           2961      1  ...    0   
2            0            0              35.0          25149      1  ...    1   
3            0            0              15.0         635296      1  ...    0   
4            0            0              49.0           2355      1  ...    0   

  alternative html mixed  plain related  signed  report  calendar  idm  
0           1    0 

In [21]:
#one hot encoding of day column

f_day = [x for x in train_df.day.value_counts().sort_values(ascending=False).index]
one_hot_top(train_df,'day',f_day)
one_hot_top(test_df,'day',f_day)
train_df.drop('day',axis=1, inplace=True)
test_df.drop('day',axis=1, inplace=True)

In [22]:
train_df = train_df.drop(columns=["date","date_new","hour",'year',''],axis=1)
test_df = test_df.drop(columns=["date","date_new","hour",'year',''],axis=1)

In [23]:
#creating a new feature using image and url

train_df["url_images"] = train_df["images"]*train_df["urls"]
train_df['url_images'].values[train_df['url_images'].values > 0] = 1


test_df["url_images"] = test_df["images"]*test_df["urls"]
test_df['url_images'].values[test_df['url_images'].values > 0] = 1

In [24]:
train_df = train_df.drop(columns=["images","urls"],axis=1)
test_df = test_df.drop(columns=["images","urls"],axis=1)

In [25]:
#one hot encoding of month

f_month = [x for x in train_df.month.value_counts().sort_values(ascending=False).index]
one_hot_top(train_df,'month',f_month)
one_hot_top(test_df,'month',f_month)
train_df.drop('month',axis=1, inplace=True)
test_df.drop('month',axis=1, inplace=True)

In [26]:
train_y = train_df[['label']]

In [27]:
#removing label from training data
train_df.drop('label',axis=1,inplace=True)

In [28]:
new_train_1 = train_df
print(new_train_1.head())

   ccs  bcced  salutations  designation  chars_in_subject  chars_in_body  \
0    0      0            0            0              49.0          80027   
1    0      0            0            0             107.0           2961   
2    0      0            0            0              35.0          25149   
3    0      0            0            0              15.0         635296   
4    1      0            0            0              49.0           2355   

  period  iiitd  usebackpack  google  ...  10  07  04  03  01  06  02  05  11  \
0      2      0            0       0  ...   0   0   0   0   0   0   0   0   1   
1      2      0            0       0  ...   0   0   0   0   0   0   1   0   0   
2      4      0            1       0  ...   0   1   0   0   0   0   0   0   0   
3      2      0            0       0  ...   1   0   0   0   0   0   0   0   0   
4      2      0            0       0  ...   0   0   0   0   0   0   0   0   1   

   12  
0   0  
1   0  
2   0  
3   0  
4   0  

[5 rows

In [29]:
#using standard scalar

from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
new_train_x = pd.DataFrame(scalar.fit_transform(new_train_1))
new_test = pd.DataFrame(scalar.transform(test_df))

#Classification

In [30]:
estimators = [('rf', RandomForestClassifier(n_estimators=18, random_state=42)),
   ('lr', make_pipeline(StandardScaler(),LogisticRegression())),          
              ('knn',make_pipeline(StandardScaler(),KNeighborsClassifier(n_neighbors=3))),
               ('xgb', xgb.XGBClassifier(objective="multi:softprob",use_label_encoder=False, random_state=42))]

level1_est = make_pipeline(StandardScaler(),svm.SVC(kernel='linear', C=0.1,random_state=42))

clf = StackingClassifier(estimators=estimators,final_estimator=level1_est,cv=5)


In [31]:
clf.fit(new_train_x,train_y).score(new_train_x,train_y)

/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

[20:25:03] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

[20:36:24] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


...

/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:49:21] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/sakshi/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:52:57] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.49670724406306127

In [32]:
pred_y = clf.predict(new_test)

In [33]:
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("/Users/sakshi/Documents/Study/ML assignment/Dec4_v7.csv", index=True, index_label='Id')